<a href="https://colab.research.google.com/github/asyraffff/eICU-UMMC-Length-Of-Stay-Prediction/blob/main/6_Streamlit_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q streamlit

In [ ]:
!pip install shap streamlit-shap

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import random
import pickle
import tensorflow as tf
import shap
from streamlit_shap import st_shap
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import matplotlib

# Load the saved model
dbn_model = tf.keras.models.load_model('drive/My Drive/FYP_LOS/dataset/dbn_model.h5')

st.title('Length of Stay Prediction for Cardiac Patient at UMMC')

if 'random_patient' not in st.session_state:
    st.session_state.random_patient = 0

if 'data' not in st.session_state:
    st.session_state.data = pd.read_csv("drive/My Drive/FYP_LOS/dataset/cardiac_ed_ppum_viva.csv", low_memory=False)

with open("drive/My Drive/FYP_LOS/dataset/dl_scaler.pkl", 'rb') as file:
    scaler = pickle.load(file)

patientID = st.selectbox(
    'Choose Patient ID',
    (0, 1, 2, 3, 4, 5, 6, 7, 8),
    index=st.session_state.random_patient)

def randomize_patient():
    st.session_state.random_patient = random.randint(0, 8)

st.button('Pick random patient', on_click=randomize_patient)

st.text(' ')
st.write("<p style='text-align: justify;'>Patient information</p>",
            unsafe_allow_html=True)
st.table(st.session_state.data.iloc[patientID])

st.write("<p style='text-align: justify;'>Diagnosis code :</p>",
            unsafe_allow_html=True)
st.markdown("**5** : Angina, unstable (angina interferes w/quality)",
            unsafe_allow_html=True)
st.markdown("**47** : Infarction, acute myocardial (MI)\nInfarction",
            unsafe_allow_html=True)
st.text(" ")

# Function to preprocess the input data similar to what was done before training the model
def preprocess_data(df):
    # If 'df' is a Series (single row of data), convert it to a DataFrame
    if isinstance(df, pd.Series):
        df = pd.DataFrame(df).transpose()

    # Create 'LOS_log' column with log-transformed LOS data
    df['LOS_log'] = np.log1p(df['LOS'])

    # Remove 'LOS' column
    X = df.drop(['LOS', 'LOS_log'], axis=1)
    st.session_state.columns = X.columns.tolist()

    X_scaled = scaler.transform(X)
    return X_scaled

# Function to predict Length of Stay (LOS) for a given patient using the DBN model
def predict_length_of_stay(patient_data):
    preprocessed_data = preprocess_data(patient_data)
    # Predict LOS using the loaded DBN model
    predicted_los = dbn_model.predict(preprocessed_data)
    return predicted_los

# Update the 'predict' function to use the DBN model for predictions
def predict():
    patient_data = st.session_state.data.iloc[patientID]
    predicted_los = predict_length_of_stay(patient_data)
    st.session_state.predicted_los = np.expm1(predicted_los)

st.button('Predict', on_click=predict)

st.subheader('Prediction result')
if 'predicted_los' in st.session_state:
    st.text(f'LOS : {st.session_state.predicted_los[0][0]:.2f} days')

if 'predicted_los' in st.session_state:
    st.title('Explainable AI (Global Interpretation)')

    st.markdown("A **global method** helps us understand the **overall structure** of **how a model makes a decision**.",
            unsafe_allow_html=True)

    st.subheader('Summary Plot')

    st.markdown("The `summary plot` shows the **most important features** and the **magnitude of their impact on the model**. It is the global interpretation",
            unsafe_allow_html=True)

    # Create an explainer using the trained DBN model and the data
    explainer = shap.DeepExplainer(dbn_model, preprocess_data(st.session_state.data))

    # Calculate SHAP values for new data
    shap_values = explainer.shap_values(preprocess_data(st.session_state.data))

    # Visualize SHAP summary plot for feature importance
    st_shap(shap.summary_plot(shap_values, features=preprocess_data(st.session_state.data),
            feature_names=st.session_state.columns))

    st.markdown("Passing a **matrix of SHAP values** to the bar plot function creates a **global feature importance plot**, where the global importance of each feature is taken to be the **mean absolute value** for that feature over all the given samples",
            unsafe_allow_html=True)

    st.markdown("Here the **features** are **ordered** from the **highest to the lowest effect on the prediction**. It takes in account the **absolute SHAP value**, so it **does not matter** if the feature affects the prediction in a **positive or negative way**.",
            unsafe_allow_html=True)

    st.subheader('Beeswarm Plot')

    st.markdown("The `beeswarm plot` is designed to display an information-dense summary of **how the top features** in a dataset **impact the model’s output**. `Each instance` the given explanation is represented by a **single dot** on each feature row.",
            unsafe_allow_html=True)

    st.markdown("The `x` position of the dot is determined by the **SHAP value** `(shap_values.value[instance,feature])` of that feature, and dots **“pile up”** along each feature row to show **density**.",
            unsafe_allow_html=True)

    st.markdown("**Color** is used to display the **original value of a feature** `(shap_values.data[instance,feature])`.",
            unsafe_allow_html=True)

    st_shap(shap.summary_plot(shap_values[0], features=preprocess_data(st.session_state.data),
                              feature_names=st.session_state.columns))

    st.markdown("In the plot above we can see that **high values** of the `intubated (1)` variable have a **high positive contribution** on the prediction, while **low values** have a **low positive contribution**.",
            unsafe_allow_html=True)

    st.markdown("The `bun` variable has a really **high positive contribution** when its values are **high**, and a **low negative contribution** on **low values**.",
            unsafe_allow_html=True)

    st.markdown("The feature `gender` has almost **no contribution** to the prediction, whether its values are high or low.",
            unsafe_allow_html=True)


    st.title('Explainable AI (Local Interpretation)')

    st.text("")
    st.subheader('Force Plot')

    st.markdown("The `force plot` is another way to see the effect each feature has on the prediction, for a given observation.",
            unsafe_allow_html=True)

    # Force plot
    st_shap(shap.force_plot(explainer.expected_value[0].numpy(),
                   shap_values[0][patientID],feature_names=st.session_state.columns,
                   out_names="LOS", figsize=(11,4), matplotlib=matplotlib))

    st.markdown("In this plot the **positive SHAP values** are displayed on the **left** side and the **negative** on the **right** side, as if competing against each other.",
            unsafe_allow_html=True)

    st.markdown("The **highlighted value** is the **prediction** for that observation.",
            unsafe_allow_html=True)

    # st.text("")
    # st.subheader('Decision Plot')

    # st.text("The decision plot makes it possible to observe the amplitude of each change,")
    # st.text("taken by a sample for the values of the displayed features.")


    # Decision plot
    # st_shap(shap.decision_plot(explainer.expected_value[0].numpy(),
                   # shap_values[0][patientID], features = preprocess_data(st.session_state.data),
                   # feature_names = st.session_state.columns))

    st.text("")
    st.subheader('Waterfall Plot')

    st.markdown("`Waterfall plots` are designed to display explanations for individual predictions.",
            unsafe_allow_html=True)

    st.markdown("The **bottom** of a waterfall plot starts as the **expected value** of the model output, and then each row shows how the **positive (red)** or **negative (blue)** contribution of each feature moves the value from the expected model output over the background dataset to the model output for this prediction.",
            unsafe_allow_html=True)

    # Waterfall plot
    st_shap(shap.plots._waterfall.waterfall_legacy(explainer.expected_value[0].numpy(),
                                       shap_values[0][patientID], feature_names = st.session_state.columns))

    st.markdown("1. **Expected Value (Top)**: The top of the plot represents the **model's expected value**. This is the **starting point** for the prediction.",
            unsafe_allow_html=True)

    st.markdown("2. **Feature Contributions**: Each **horizontal bar** represents the **contribution** of a specific feature to the prediction.",
            unsafe_allow_html=True)

    st.markdown("3. **Positive Contribution (+)**: If a bar extends to the **right** of the expected value, it indicates a **positive contribution**. For example, `hematocrit` has a positive contribution of `0.07`. This means that the **presence of hematocrit increases the model's prediction** by `0.07`.",
            unsafe_allow_html=True)

    st.markdown("4. **Negative Contribution (-)**: If a bar extends to the **left** of the expected value, it indicates a **negative contribution**. For example, `intubated` has a negative contribution of `0.07`. This means that the **presence of intubated decreases the model's prediction** by `0.07`.",
            unsafe_allow_html=True)

    st.markdown("5. **Cumulative Sum**: As you **move down** the plot, the contributions are **added** to the cumulative sum.",
            unsafe_allow_html=True)

    st.markdown("6. **Final Prediction**: The **final position** on the plot represents the **model's prediction** for the specific instance.",
            unsafe_allow_html=True)

    st.markdown("In summary, **positive contributions push the prediction higher**, and **negative contributions pull it lower**. The waterfall plot provides a clear visual representation of how each feature's contribution combines to form the model's prediction for a specific instance.",
            unsafe_allow_html=True)

Overwriting app.py


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.601s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
!streamlit run app.py --server.address=localhost &>/content/logs.txt &

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 35.199.154.13


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 3.152s
your url is: https://cuddly-weeks-leave.loca.lt
